In [96]:
#Importing packages and data

import pandas as pd
import numpy as np
from sklearn import linear_model, metrics, preprocessing
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,accuracy_score
from sklearn.metrics import confusion_matrix,accuracy_score

seeds = pd.read_csv('seeds_dataset.txt', delimiter='\t', header = None)
num_row, num_col = seeds.shape
seeds.head(10)

,0,1,2,3,4,5,6,7
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1
5,14.38,14.21,0.8951,5.386,3.312,2.462,4.956,1
6,14.69,14.49,0.8799,5.563,3.259,3.586,5.219,1
7,14.11,14.10,0.8911,5.420,3.302,2.700,5.000,1
8,16.63,15.46,0.8747,6.053,3.465,2.040,5.877,1
9,16.44,15.25,0.8880,5.884,3.505,1.969,5.533,1


In [97]:
### Splitting train and test
train = seeds.sample(frac=0.8, random_state=25)
test = seeds.drop(train.index)

C0 = seeds.copy()
C1 = seeds.copy()
C2 = seeds.copy()

training_data_set = [C0, C1, C2]

In [98]:
# print(training_data_set)
# len(training_data_set)
# print(training_data_set[1])
# print(training_data_set[i].at[4,7])

for i in range(3):
    for j in range(210):
        if training_data_set[i].at[j,7] == i:
#             print("i match: ", training_data_set[i].at[j,7])
            training_data_set[i].at[j,7] = 1
        else:
#             print("i did not match: ", training_data_set[i].at[j,7])
            training_data_set[i].at[j,7] = 0
            
     

In [100]:
datadict = {}  

for i in range(3):
    X = training_data_set[i].iloc[:,:7]
    # min max scaling
    min_max_scaler = preprocessing.MinMaxScaler()
    X = min_max_scaler.fit_transform(X)
    X = X.T
    y = training_data_set[i].iloc[:,-1]
    # y label encoding
    lab = preprocessing.LabelEncoder()
    y = lab.fit_transform(y)
    y = np.array([y])
    datadict['C'+str(i)] = [X,y]  

In [101]:
itterations = 100000   #random big value 
alpha = 0.160 
m = 210
cost_values = []
trained_parameters = []

for trainsets in range(3):
  X = datadict['C'+str(trainsets)][0] 
  y = datadict['C'+str(trainsets)][1]
  weights = np.random.randn(1,7)
  bias = 0 
  costfunc_values = []
  k = 0
  print('Training for dataset '+str(trainsets))
  for i in range(1,itterations+1):
    #logistic function
    try: 
        z = np.dot(weights,X) + bias 
        hypothesis = 1/(1 + np.exp(-z)) 
        #cost function
        j = 1/m*(-1*(np.sum(y*np.log(hypothesis) + (1-y)*np.log(1-hypothesis))))
        costfunc_values.append(j)
        k+=1
        #gradient decent
        dw =  1/m * np.dot(hypothesis-y,X.T)
        db =  1/m * np.sum(hypothesis-y)
        weights = weights - alpha*dw
        bias = bias - alpha*db
        if i%10 == 0:
            if abs(j-costfunc_values[i-2])<0.000001:
                if abs(j-costfunc_values[i-3])<0.000001:
                    break 
    except:
        continue
  cost_values.append(costfunc_values)    
  trained_parameters.append([weights,bias])
  print("total iterations: ", i)
  print("weight and bias for dataset", trainsets)
  print(trained_parameters[trainsets])

    
 

Training for dataset 0
total iterations:  2830
weight and bias for dataset 0
[array([[-2.51374443, -0.85196054, -1.52723908, -0.31941916, -1.87426562,
        -1.32041136,  0.14106833]]), -3.891952690082886]
Training for dataset 1
total iterations:  69750
weight and bias for dataset 1
[array([[ -4.27635418,   6.87350007,   4.09711272,  24.8793411 ,
         -4.19071771,  -6.84636963, -32.82384848]]), 3.443587082903393]
Training for dataset 2
total iterations:  28990
weight and bias for dataset 2
[array([[ 4.81629084,  5.75340267, -1.5776131 , -1.86042682,  5.09029618,
         4.80722565, 10.96756825]]), -13.993467305563486]


In [102]:
for datasetnum in range(3):
  X = datadict['C'+str(datasetnum)][0]
  y = datadict['C'+str(datasetnum)][1]
  weights = trained_parameters[datasetnum][0]
  bias = trained_parameters[datasetnum][1]
  correct_predictions = 0 
  for i in range(210):
    z = np.dot(weights,X.T[i,:])+bias
    hypothesis = 1/(1 + np.exp(-z))
    if np.logical_and(hypothesis >= 0.5,y.T[i,0] == 1):
        correct_predictions+=1
    if np.logical_and(hypothesis < 0.5,y.T[i,0] == 0):
        correct_predictions+=1    
  #print(correct_predictions)      
  acc = (correct_predictions/210)*100
  print('accuracy for dataset '+str(datasetnum)," = ",acc)

accuracy for dataset 0  =  100.0
accuracy for dataset 1  =  95.23809523809523
accuracy for dataset 2  =  98.09523809523809
